In [1]:
%reset -f
import gc
gc.collect()

0

In [2]:
import numpy as np
import pandas as pd
import csv
import os
import datetime
from opencc import OpenCC
import re
import chardet
import random
import time
import json
from tqdm import trange
import requests
from bs4 import BeautifulSoup

## 爬取基金類別績效

In [23]:
url_params = {'txtYEAR':'2021', 'txtMONTH':'01'}
url = "https://www.sitca.org.tw/ROC/Industry/IN2421.aspx"
r = requests.get(url,params = url_params)
r.encoding = "utf8"
print(r)
soup = BeautifulSoup(r.text, "lxml")
Trs = soup.find_all("tr", attrs={"class":["DTeven","DTodd"]})

<Response [200]>


In [24]:
url

'https://www.sitca.org.tw/ROC/Industry/IN2421.aspx'

In [25]:
pattern = "EUCA\w+"
df = []
columnName = ['基金代號','基金名稱','1個月','3個月','6個月','1年','2年','3年','5年','累計','十年','年化標準差','Sharpe']
for tr in Trs:
    tds = tr.find_all("td")
    tmp=[]
    for idx in range(len(tds)):
        if(idx == 0):
            tmp.append(re.findall(pattern, tds[idx].a["href"])[0])
        if(tds[idx].string.strip()==''):
            tmp.append(np.nan)
        else:
            tmp.append(tds[idx].string.strip())
    df.append(tmp)
df = pd.DataFrame(df, columns = columnName)
df.head(10)

,基金代號,基金名稱,1個月,3個月,6個月,1年,2年,3年,5年,累計,十年,年化標準差,Sharpe
0,EUCA000500,大中華股票,6.79,20.64,26.22,51.63,79.92,40.17,142.26,6.79,116.27,19.77,0.59
1,EUCA000501,亞太區不包括日本股票,3.61,21.5,26.75,32.76,42.06,21.96,102.29,3.61,87.5,18.78,0.37
2,EUCA000502,亞太區股票,1.65,18.28,23.6,23.2,31.9,12.03,73.14,1.65,65.94,16.63,0.2
3,EUCA000503,澳洲及紐西蘭股票,0,22.57,21.89,15.23,29.76,14.68,70.97,0,38.49,24.13,0.25
4,EUCA000504,中國股票,6.41,19.08,25.9,52.87,67.82,30.49,130.84,6.41,94.61,19.47,0.48
5,EUCA000507,全球新興市場股票,3.15,21.61,24.61,26.66,32.73,11.75,90.72,3.15,39.4,19.76,0.21
6,EUCA000508,新興歐洲股票,0,25.41,10.61,-14.53,2.22,-5.3,42.27,0,-8.16,26.33,-0.04
7,EUCA000509,歐洲不包括英國大型股票,-0.71,16.42,12.06,3.59,22.14,9.95,38.94,-0.71,104.94,20.41,0.16
8,EUCA000510,歐洲不包括英國中小型股票,0.36,21.7,22.45,16.16,33.3,17.8,62.44,0.36,161.65,23.08,0.22
9,EUCA000511,歐洲大型均衡型股票,-0.61,15.21,11.11,-0.71,14.74,4.32,24.96,-0.61,65.14,19.45,0.06


## 爬取基金個別列表

In [26]:
url_params = {'txtYEAR':'2019', 'txtMONTH':'01', 'txtGROUPID':'EUCA000500'}
url_headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"}
url = "https://www.sitca.org.tw/ROC/Industry/IN2422.aspx"

r = requests.get(url, headers = url_headers, params = url_params)
r.encoding = "utf8"
soup = BeautifulSoup(r.text, "lxml")
table = soup.find(id="ctl00_ContentPlaceHolder1_TableClassList")
trs = table.find_all("tr")
newTrs = trs[1].find("table").find_all("tr")

columnName = ['基金名稱','基金英文名','1個月','3個月','6個月','1年','2年','3年','5年','累計','十年','年化標準差','Sharpe']
df = []
for i in range(2, len(newTrs)):
    tds = newTrs[i].find_all("td")
    tmp=[]
    for idx in range(1, len(tds)):
        if(tds[idx].string.strip()==''):
            tmp.append(np.nan)
        else:
            tmp.append(tds[idx].string.strip())            
    df.append(tmp)
df = pd.DataFrame(df, columns = columnName)
df['1個月'] = pd.to_numeric(df['1個月'])
df['3個月'] = pd.to_numeric(df['3個月'])
df['6個月'] = pd.to_numeric(df['6個月'])
df['1年'] = pd.to_numeric(df['1年'])
df['2年'] = pd.to_numeric(df['2年'])
df['3年'] = pd.to_numeric(df['3年'])
df['5年'] = pd.to_numeric(df['1個月'])
df['累計'] = pd.to_numeric(df['累計'])
df['十年'] = pd.to_numeric(df['十年'])
df['年化標準差'] = pd.to_numeric(df['年化標準差'])
df['Sharpe'] = pd.to_numeric(df['Sharpe'])
df.head(10)

,基金名稱,基金英文名,1個月,3個月,6個月,1年,2年,3年,5年,累計,十年,年化標準差,Sharpe
0,景順大中華基金A股 美元,Invesco Greater China Equity A USD Acc,5.94,3.62,-10.96,-15.47,21.39,51.05,5.94,5.94,223.19,14.31,0.95
1,瑞銀 (盧森堡) 大中華股票基金 (美元),UBS (Lux) EF Greater China (USD) P,11.02,13.08,-5.19,-14.08,41.03,69.68,11.02,11.02,282.55,16.49,1.08
2,富蘭克林坦伯頓全球投資系列-大中華基金美元A (acc)股,Templeton China A(acc)USD,7.79,7.10,-5.90,-11.76,22.91,53.04,7.79,7.79,120.64,14.65,0.96
3,NN (L) 大中華股票基金P股美元,NN (L) Greater China Equity P Cap USD,7.49,9.34,-3.81,-12.39,19.13,40.61,7.49,7.49,174.66,13.86,0.80
4,富達基金-大中華基金,Fidelity Greater China A-Dis-USD,9.20,9.87,-6.78,-16.56,28.05,59.01,9.20,9.20,247.64,15.95,0.98
5,施羅德環球基金系列－大中華(美元)A1-累積,Schroder ISF Greater China A1 Acc USD,6.98,8.28,-6.75,-14.77,27.04,61.19,6.98,6.98,205.07,15.38,1.04
6,施羅德環球基金系列－大中華(美元)A-累積,Schroder ISF Greater China A Acc USD,7.03,8.42,-6.52,-14.34,28.32,63.63,7.03,7.03,219.49,15.38,1.07
7,施羅德環球基金系列－大中華(美元)C-累積,Schroder ISF Greater China C Acc USD,7.08,8.57,-6.26,-13.87,29.74,66.38,7.08,7.08,240.10,15.39,1.10
8,摩根基金-JPM大中華(美元)-A股(分派),JPM Greater China A (dist) USD,9.49,10.57,-11.88,-23.11,22.53,45.50,9.49,9.49,191.29,17.84,0.72
9,NN (L) 大中華股票基金X股美元,NN (L) Greater China Equity X Cap USD,7.45,9.20,-4.05,-12.83,17.96,38.54,7.45,7.45,161.65,13.85,0.77
